In [ ]:
!pip install llama-index

In [ ]:
import os
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)

### Set Up Api Key
- Navigate to the “Secrets” section in the sidebar.
- Click on “Add a new secret.”
- Enter the name and value of the secret. name=OPENAI_API_KEY, value=<api_key>


In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### Load Data

### Load Data Files

In [ ]:
filename_fn = lambda filename: {"file_name": filename}
# automatically sets the metadata of each document according to filename_fn
documents = SimpleDirectoryReader(
    input_files=["./data/01.pdf","./data/02.pdf","./data/03.pdf","./data/04.pdf","./data/05.pdf"], file_metadata=filename_fn
).load_data()

In [ ]:
print(f"Loaded {len(documents)} docs")

Loaded 91 docs


### Save Indexing

In [ ]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
  # rebuild storage context
  storage_context = StorageContext.from_defaults(persist_dir="storage")
  # load index
  index = load_index_from_storage(index_id="vector_index", storage_context=storage_context)

### Retrieval

In [ ]:
# import openai
# !openai migrate
# openai.api_key = 'YOUR_API_KEY'
# messages = [ {"role": "system", "content":
#               "You are a intelligent assistant."} ]
# while True:
#     message = input("User : ")
#     if message:
#         messages.append(
#             {"role": "user", "content": message},
#         )
#         chat = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo", messages=messages
#         )
#     reply = chat.choices[0].message.content
#     print(f"ChatGPT: {reply}")
#     messages.append({"role": "assistant", "content": reply})

In [ ]:
query_engine = index.as_query_engine(response_mode="tree_summarize", similarity_top_k=5)

In [ ]:
response = query_engine.query(""" Explain in 5000 words like a doctor explaining to patient. Add more details based on general medical knowledge.

EXAM: MR LUMBAR SPINE WITH AND WITHOUT CONTRAST

CLINICAL INDICATION: Low back pain.

TECHNIQUE: Multiplanar multisequence MRI of the lumbar spine performed without and with intravenous
contrast.

CONTRAST: 15 mL of Clariscan

COMPARISON: None.

FINDINGS:
Assuming normal segmentation, there are 5 lumbar type vertebra with S1 completely incorporated into the
sacrum.

Postsurgical changes of L2-L4 laminectomies with posterior instrumented fusion at L2-L5. Susceptibility
artifact related to fusion hardware limits evaluation of adjacent structures.

Vertebral Bodies: Chronic-appearing mild anterior wedging of the T12 vertebral body. Vertebral body
heights are otherwise maintained. No suspicious marrow signal replacement.
Alignment: No acute lumbar subluxation. Stepwise grade 1 retrolisthesis of L1 on L2 and L2 on L3. Mild
lumbar dextrocurvature.

Extradural: There are no abnormal extradural fluid collections or masses. No foci of abnormal
enhancement.
Spinal Cord: The distal spinal cord appears normal.
Conus: The conus terminates at approximately L1

T12-L1: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L1-L2: Severe spinal canal stenosis due to disc bulge, facet arthropathy and ligamentum flavum
redundancy. Moderate right and severe left neuroforaminal stenosis due to foraminal extension of disc
bulge and facet arthropathy.

L2-L3: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L3-L4: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L4-L5: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L5-S1: No spinal canal stenosis. Moderate right and severe left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

Paraspinal Soft Tissues: Unremarkable.
Regional Soft Tissues: Simple-appearing right renal cyst measuring 6.7 x 5.8 cm.


IMPRESSION:

Postsurgical and multilevel degenerative spondylotic changes of the lumbar spine with up to severe spinal
canal and severe neuroforaminal stenosis, further discussed in level by level detail above.
""")

In [ ]:
response.response

'The MRI of your lumbar spine shows several findings. Firstly, the lumbar vertebrae are normal in their alignment and segmentation, with the sacrum fully incorporated into the S1 vertebra. There are postsurgical changes noted at the L2-L4 levels, where laminectomies were performed with posterior instrumented fusion at L2-L5. However, the evaluation of adjacent structures is limited due to artifact related to the fusion hardware.\n\nIn terms of the vertebral bodies, there is a chronic-appearing mild anterior wedging of the T12 vertebral body, but overall the vertebral body heights are maintained with no suspicious marrow signal replacement. The alignment of the spine shows no acute subluxation, but there is a mild dextrocurvature present. Additionally, there is stepwise grade 1 retrolisthesis of L1 on L2 and L2 on L3.\n\nMoving on to the specific levels of the lumbar spine:\n- At the T12-L1 level, there is no spinal canal stenosis, but there is mild bilateral neuroforaminal stenosis due

In [ ]:
#explore which nodes were selected for the answer
for node_with_score in response.source_nodes:
    print(node_with_score.node)

Node ID: 685c27c8-3041-4c7c-b2e6-2cc98404fd63
Text: noninfectious inﬂammatory processes. MRI is the test of choice,
as it can show not only cord compression but also intramedullary
pathology. SagittalT2 imaging supplemented by sagittal proton density,
STIR, and axial T2 im-aging is essential for diagnosis of
intramedullary signal abnormality ( Fig. 2 ). Pyogenic cervical
spondylodiskitis Pyogenic...
Node ID: 6313fa69-7955-4c9b-9486-cd4d038bdff7
Text: pathologic compression fractures [149–152] ; MRI and multiplanar
CT are helpful in preoperative planning [153]. The postoperative spine
The United States has the highest incidence of spine surgery in the
world, and it has increased over the past 2 decades [154]. There is an
incidence of approximately 15% to 20% of failures, however, termed
faile...
Node ID: 4470db61-3176-476e-9631-14ee92ce88d0
Text: vertebral bodies.16However, with exaggerated extension, 1- to
2-mm budging may occur in some histologically normal discs.17–19The
pos- terior 

## To Do
- Actual Data Ingestion
- Evaluate baseline
- Improve Perf

EXAM: MR LUMBAR SPINE WITH AND WITHOUT CONTRAST

CLINICAL INDICATION: Low back pain.

TECHNIQUE: Multiplanar multisequence MRI of the lumbar spine performed without and with intravenous
contrast.

CONTRAST: 15 mL of Clariscan

COMPARISON: None.

FINDINGS:
Assuming normal segmentation, there are 5 lumbar type vertebra with S1 completely incorporated into the
sacrum.

Postsurgical changes of L2-L4 laminectomies with posterior instrumented fusion at L2-L5. Susceptibility
artifact related to fusion hardware limits evaluation of adjacent structures.

Vertebral Bodies: Chronic-appearing mild anterior wedging of the T12 vertebral body. Vertebral body
heights are otherwise maintained. No suspicious marrow signal replacement.
Alignment: No acute lumbar subluxation. Stepwise grade 1 retrolisthesis of L1 on L2 and L2 on L3. Mild
lumbar dextrocurvature.

Extradural: There are no abnormal extradural fluid collections or masses. No foci of abnormal
enhancement.
Spinal Cord: The distal spinal cord appears normal.
Conus: The conus terminates at approximately L1

T12-L1: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L1-L2: Severe spinal canal stenosis due to disc bulge, facet arthropathy and ligamentum flavum
redundancy. Moderate right and severe left neuroforaminal stenosis due to foraminal extension of disc
bulge and facet arthropathy.

L2-L3: No spinal canal stenosis. Mild bilateral neuroforaminal stenosis due to foraminal extension of
disc bulge and facet arthropathy.

L3-L4: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L4-L5: No spinal canal stenosis. Mild right and moderate left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

L5-S1: No spinal canal stenosis. Moderate right and severe left neuroforaminal stenosis due to foraminal
extension of disc bulge and facet arthropathy.

Paraspinal Soft Tissues: Unremarkable.
Regional Soft Tissues: Simple-appearing right renal cyst measuring 6.7 x 5.8 cm.


IMPRESSION:

Postsurgical and multilevel degenerative spondylotic changes of the lumbar spine with up to severe spinal
canal and severe neuroforaminal stenosis, further discussed in level by level detail above.